In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/t3/8kjgwj5916lcv_26jt1s3fg80000gn/T/ipykernel_24596/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [5]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [30]:
# Your code
from sklearn.model_selection import train_test_split
data_train, data_val = train_test_split(data, test_size=0.3, random_state=42, stratify=data['label'])

## Data Preprocessing

In [31]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
#print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [32]:
from bs4 import BeautifulSoup

In [42]:
# Your code

def clean_html(html_text):
    soup = BeautifulSoup(html_text, "html.parser")
    return soup.get_text()

data_train['preprocessed_text'] = data_train['text'].apply(clean_html)
data_val['preprocessed_text'] = data_val['text'].apply(clean_html)

/var/folders/t3/8kjgwj5916lcv_26jt1s3fg80000gn/T/ipykernel_24596/1424907659.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html_text, "html.parser")
/var/folders/t3/8kjgwj5916lcv_26jt1s3fg80000gn/T/ipykernel_24596/1424907659.py:4: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(html_text, "html.parser")


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [43]:
# Your code
import re
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# Initialize the SnowballStemmer
snowball = SnowballStemmer('english')

# Function for text preprocessing
def preprocess_text(text):
    # 1. Remove all special characters
    text = re.sub(f"[{re.escape(string.punctuation)}]", " ", text)

    # 2. Remove numbers
    text = re.sub(r'\d+', '', text)

    # 3. Remove single characters (isolated single letters)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

    # 4. Remove single characters from the start
    text = re.sub(r'\b[a-zA-Z]\b', '', text)

    # 5. Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)

    # 6. Remove prefixed 'b' (handling any byte-like artifacts)
    text = re.sub(r'^b\s+', '', text)

    # 7. Convert to lowercase
    text = text.lower()

    return text

# Example usage on X_train (assuming X_train is a list or pandas Series of text)
data_train['preprocessed_text'] = data_train['preprocessed_text'].apply(preprocess_text)
data_val['preprocessed_text'] = data_val['preprocessed_text'].apply(preprocess_text)

# Output one of the cleaned texts to verify
print(data_train[:5])


                                                  text  label  \
381  Mills Cheryl D <MillsCD@state.gov>Sunday Janua...      0   
428  H <hrod17@clintonemail.com >Saturday January 2...      0   
849  DEAR,     MY NAME IS MR MR Ken Edward,A former...      1   
252  Dear Sir, I am Engr. Victor Chigoziem with the...      1   
380  Hello,This is Dr.Clive Whittaker. I work for F...      1   

                                     preprocessed_text  
381  mills cheryl sunday january pmtravel schedulec...  
428   saturday january pm sbwhoeopre fyi any foreig...  
849  dear my name is mr mr ken edward former govern...  
252  dear sir am engr victor chigoziem with the eng...  
380  hello this is dr clive whittaker work for fide...  


## Now let's work on removing stopwords
Remove the stopwords.

In [44]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
# Load the stopwords
stop_words = set(stopwords.words('english'))

# Function to remove stopwords from a list of words
def remove_stopwords(text):
    # Tokenize the text (split into words)
    words = text.split()
    # Remove stopwords
    filtered_words = [word for word in words if word not in stop_words]
    # Join the words back into a single string
    return ' '.join(filtered_words)

# Example usage on X_train (assuming X_train is a list or pandas Series of text)
data_train['preprocessed_text'] = data_train['preprocessed_text'].apply(remove_stopwords)
data_val['preprocessed_text'] = data_val['preprocessed_text'].apply(remove_stopwords)

# Output one of the stopword-removed texts to verify
print(data_train[:5])


                                                  text  label  \
381  Mills Cheryl D <MillsCD@state.gov>Sunday Janua...      0   
428  H <hrod17@clintonemail.com >Saturday January 2...      0   
849  DEAR,     MY NAME IS MR MR Ken Edward,A former...      1   
252  Dear Sir, I am Engr. Victor Chigoziem with the...      1   
380  Hello,This is Dr.Clive Whittaker. I work for F...      1   

                                     preprocessed_text  
381  mills cheryl sunday january pmtravel schedulec...  
428  saturday january pm sbwhoeopre fyi foreign nat...  
849  dear name mr mr ken edward former government o...  
252  dear sir engr victor chigoziem engineering sto...  
380  hello dr clive whittaker work fidelity investm...  


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/weemoe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [45]:
# Your code
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download the necessary NLTK resources
nltk.download('punkt_tab')  # This is needed for tokenization
nltk.download('wordnet')
nltk.download('omw-1.4')
# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Function to lemmatize text
def lemmatize_text(text):
    # Tokenize the text (split into words)
    words = word_tokenize(text)
    # Lemmatize each word
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    # Join the words back into a single string
    return ' '.join(lemmatized_words)

# Apply lemmatization to X_train_no_stopwords
data_train['preprocessed_text'] = data_train['preprocessed_text'].apply(lemmatize_text)
data_val['preprocessed_text'] = data_val['preprocessed_text'].apply(lemmatize_text)

# Output the lemmatized text to verify
print(data_train['preprocessed_text'][:5])

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/weemoe/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/weemoe/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/weemoe/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


381    mill cheryl sunday january pmtravel scheduleca...
428    saturday january pm sbwhoeopre fyi foreign nat...
849    dear name mr mr ken edward former government o...
252    dear sir engr victor chigoziem engineering sto...
380    hello dr clive whittaker work fidelity investm...
Name: preprocessed_text, dtype: object


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [46]:
from collections import Counter

# Initialize the bag of words Counter
bag_of_words = Counter()

# Loop over each text in X_train_no_stopwords
for text in data_train['preprocessed_text']:
    # Split the text into words
    words = text.split(' ')
    # Update the Counter with the words
    bag_of_words.update(words)

# Get the 10 most common words
top_10_words = bag_of_words.most_common(10)

# Display the top 10 words and their counts
print(top_10_words)


[('money', 634), ('account', 603), ('bank', 562), ('u', 556), ('fund', 534), ('mr', 405), ('country', 383), ('transaction', 377), ('business', 367), ('million', 327)]


## Extra features

In [58]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

,text,label,preprocessed_text,money_mark,suspicious_words,text_len
381,Mills Cheryl D <MillsCD@state.gov>Sunday Janua...,0,mill cheryl sunday january pmtravel scheduleca...,1,0,103
428,H <hrod17@clintonemail.com >Saturday January 2...,0,saturday january pm sbwhoeopre fyi foreign nat...,1,0,116
849,"DEAR, MY NAME IS MR MR Ken Edward,A former...",1,dear name mr mr ken edward former government o...,1,1,900
252,"Dear Sir, I am Engr. Victor Chigoziem with the...",1,dear sir engr victor chigoziem engineering sto...,1,1,2252
380,"Hello,This is Dr.Clive Whittaker. I work for F...",1,hello dr clive whittaker work fidelity investm...,1,1,858


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [60]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Fit the vectorizer on the training data
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(data_train['preprocessed_text'])

# Transform the validation data using the trained vectorizer
X_val = vectorizer.transform(data_val['preprocessed_text'])

# Convert these into DataFrames to inspect
bow_df_train = pd.DataFrame(X_train.toarray(), columns=vectorizer.get_feature_names_out())
bow_df_val = pd.DataFrame(X_val.toarray(), columns=vectorizer.get_feature_names_out())




## And the Train a Classifier?


In [63]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC


# Extract features and labels from training and validation sets
X_train = bow_df_train
y_train = data_train['label']
X_val = bow_df_val
y_val = data_val['label']

# Initialize a Logistic Regression classifier

classifier_dict = {'LogisticRegression':LogisticRegression(),
                   'MultinomialNB':MultinomialNB(),
                   'SVC':SVC()}
for key, classifier in classifier_dict.items():

    # Train the classifier on the training data
    classifier.fit(X_train, y_train)

    # Predict on the validation data
    y_pred = classifier.predict(X_val)

    # Evaluate the classifier's performance
    accuracy = accuracy_score(y_val, y_pred)
    print(f'Validation Accuracy of the {key} is: {accuracy}')

Validation Accuracy of the LogisticRegression is: 0.9733333333333334
Validation Accuracy of the MultinomialNB is: 0.95
Validation Accuracy of the SVC is: 0.9333333333333333


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code